In [ ]:
import pandas as pd


In [ ]:
# Load survey data
file_path = 'data.xlsx'  # Ensure data.xlsx is in the same directory
index2 = pd.read_excel(file_path, sheet_name='index_2')
df = pd.read_excel(file_path, sheet_name='Pollfish_Survey_Work_behaviours')


In [ ]:
# Identify cognitive inputs (I), task questions (Q47–Q76), and performance targets (T)
input_codes = index2[index2['xy'] == 'I']['Code'].tolist()
task_codes = [code for code in index2['Code'] 
              if code.startswith('Q') and 47 <= int(code[1:]) <= 76 and code in df.columns]
target_codes = index2[index2['xy'] == 'T']['Code'].tolist()

# Map Q85 work positions into broader job groups
def map_position(pos):
    p = str(pos).lower()
    if 'engineer' in p: return 'Engineering'
    if 'manager' in p or 'director' in p or 'lead' in p: return 'Management'
    if 'sales' in p: return 'Sales'
    if 'marketing' in p: return 'Marketing'
    if 'hr' in p or 'human resources' in p: return 'HR'
    if 'design' in p: return 'Design'
    if 'analyst' in p: return 'Analytics'
    if 'admin' in p or 'assistant' in p: return 'Administration'
    return 'Other'

df['JobGroup'] = df['Q85'].apply(map_position)

print(f"Inputs: {len(input_codes)} codes, Tasks: {len(task_codes)} codes, Targets: {len(target_codes)} codes")


In [ ]:
# Compute and export correlation matrices for each job group
output_file = 'correlations_by_jobgroup_with_tasks.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for group, subset in df.groupby('JobGroup'):
        # Combine input and task codes as rows
        rows = input_codes + task_codes
        # Ensure numeric data
        data = subset[rows + target_codes].select_dtypes(include='number')
        # Compute correlation between rows (inputs+tasks) and targets
        corr_matrix = data[rows].corrwith(data[target_codes].mean(axis=1))  # but this gives a series
        # Instead compute full matrix: corr of each row code vs each target code
        corr_df = data[rows + target_codes].corr().loc[rows, target_codes]
        # Write to sheet
        sheet_name = group[:31]  # sheet name limit
        corr_df.to_excel(writer, sheet_name=sheet_name)
print(f"Exported excel to {output_file}")